# getting started

To succesfully run this notebook, you need:

- A NASA Earthdata login (go here: https://urs.earthdata.nasa.gov/ and click **REGISTER** to sign up if you haven't already done so)
- A `.netrc` file in your `$HOME` directory that contains your NASA Earthdata credentials. For more detailed instructions on how to set this up, see this guide: https://iamdonovan.github.io/teaching/egm722/practicals/earthaccess.html#the-netrc-file

Once you have that (and assuming that you have a conda environment set up based on the `environment.yml` file in this repository), you should be able to search and download snow cover data for a given basin by editing the code below to select the desired basin outline and dataset.

In [ ]:
from pathlib import Path
import earthaccess
import geopandas as gpd
import rasterio as rio
import shapely
from unidecode import unidecode

First, load the shapefile of basin outlines and show the available basin names:

In [ ]:
basins = gpd.read_file('../basins.gpkg').set_index('name')
list(basins.index) # show the available basin names

Next, select the basin outline by one of the names above, then prepare a search area based on the basin outline:

In [ ]:
# select the basin outline to search using the name (e.g., Copiapó)
basin_name = 'Copiapó'
basin = basins.loc[basin_name, 'geometry']

# eventually, this step won't be needed
search_area = shapely.geometry.polygon.orient(basin.minimum_rotated_rectangle, sign=1)

basin # show the outline of the basin

Next, log in to `earthaccess` using your `.netrc` credentials:

In [ ]:
earthaccess.login(strategy='netrc')

NASA Earthdata have the following daily, "cloud-free" snow cover datasets from either MODIS or VIIRS:

| Dataset                                                       | Short Name |
| ------------------------------------------------------------- | ---------- |
| MODIS/Terra CGF Snow Cover Daily L3 Global 500m SIN Grid V061 | MOD10A1F   |
| MODIS/Aqua CGF Snow Cover Daily L3 Global 500m SIN Grid V061  | MYD10A1F   |
| VIIRS/JPSS1 CGF Snow Cover Daily L3 Global 375m SIN Grid V002 | VJ110A1F   |
| VIIRS/NPP CGF Snow Cover Daily L3 Global 375m SIN Grid V002   | VNP10A1F   |

To search for a particular dataset, replace **REPLACE** in the cell below with the corresponding short name from the table above:

In [ ]:
ds_name = 'REPLACE' # add the dataset short name from above to search for granules that intersect the basin of interest

results = earthaccess.search_data(
    short_name=ds_name,
    polygon=search_area.exterior.coords
)

Finally, run the cell below to download the results for the given basin to a folder of the form `{name}_{dataset}` (e.g., `copiapo_MOD10A1F` for the MODIS/Terra CGF Snow Cover Daily L3 Global 500m SIN Grid V061 (MOD10A1F) granules that cover the Copiapó basin):

**NOTE/WARNING**: chances are good that there are a lot of results for any given basin (the datasets have daily resolution, after all), so make sure that you have sufficient disk space, a stable internet connection, and time to let the computer sit and download for a while!

In [ ]:
data_directory = '.' # replace this path with the base directory you want to download the files to
downloaded_files = earthaccess.download(results, Path(data_directory, f"{unidecode(basin_name)}_{ds_name}"))